will need to clip the zip codes to make sure we're only looking at the right zips (geopandas geometry relations https://geopandas.org/getting_started/introduction.html#Geometry-relations)

they are not cleaning the files for us. pandas will help with that, uku says.

going to need to remove or deal with the descriptive column and the non-descriptive column that just has codes - might be better to rename them to something easier?
you don't have to delete the second row in pandas. pandas.pydata.org/docs/reference/api/pandas.read_csv.html
- you can use header to tell it to use the first row as column names. this should be the top row, the non-descript row.
- use skiprows to skip the second row. it's an awful row.

make sure you're dealing with the estimate, not the margin of error. but look at the margin of error to see if its too big.

don't do any cleaning in excel. if you do, mention in readme that you did pre-processing.

but just bring it in and deal with it in pandas. it's more reproducible that way.

coastal features will help us create our own base map if we want to. but we'd probably have to mask out the water.

be sure to download something about bike lanes from the mass.gov stuff https://docs.digital.mass.gov/dataset/massgis-data-layers

be sure you're using massachussets state plane. mass.gov stuff will come in like that, but the census data probably won't. don't forget to set that.

open street map can help bring in restaurants!

contextily lets you bring in basemaps 

